## Required code for ResNet 

In [3]:
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm1d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
    


## Implementing FirstLayersBlock

In [4]:
from prixfixe.prixfixe import FirstLayersBlock

class WrongResnetFirstLayersBlock(FirstLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         out_channels=out_channels,
                         seqsize=seqsize)
        kernel_size = 7
        stride = 1
        padding = "same"

        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels,
                                  out_channels,
                                  kernel_size = kernel_size,
                                  stride = stride, 
                                  padding = padding),
                        nn.BatchNorm1d(64),
                        nn.ReLU())

In this scenario, we failed to implement several methods required for `FirstLayersBlock`. Abstract class won't allow us to use such a module


In [5]:
WrongResnetFirstLayersBlock(in_channels=4,
                       out_channels=64, 
                       seqsize=110)

TypeError: Can't instantiate abstract class WrongResnetFirstLayersBlock with abstract method forward

In [6]:
from prixfixe.prixfixe import FirstLayersBlock

class ResnetFirstLayersBlock(FirstLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         out_channels=out_channels,
                         seqsize=seqsize)
        kernel_size = 7
        stride = 1
        padding = "same"

        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels,
                                  out_channels,
                                  kernel_size = kernel_size,
                                  stride = stride, 
                                  padding = padding),
                        nn.BatchNorm1d(64),
                        nn.ReLU())
    def forward(self, x):
        x = self.conv1(x)
        return x

In [7]:
ResnetFirstLayersBlock(in_channels=4,
                       out_channels=64, 
                       seqsize=110)

ResnetFirstLayersBlock(
  (conv1): Sequential(
    (0): Conv1d(4, 64, kernel_size=(7,), stride=(1,), padding=same)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)

## Implementing CoreBlock

In [8]:
from prixfixe.prixfixe import CoreBlock

class ResNetCoreBlock(CoreBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
              out_channels=out_channels,
              seqsize=seqsize)
        block =  ResidualBlock
        layers =  [3,4,6,3]
        self.inplanes = 64
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, out_channels, layers[3], stride = 2)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [9]:
ResNetCoreBlock(in_channels=64, 
               out_channels=128,
               seqsize=110)

ResNetCoreBlock(
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (2): Resi

## Implementing FinalLayersBlock

In [10]:
from prixfixe.prixfixe import FinalLayersBlock

class WrongResNetFinalLayersBlock(FinalLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         seqsize=seqsize)
        
        self.avgpool = nn.AvgPool1d(1) # global average pooling
        self.linear = nn.Sequential(
              nn.Linear(in_channels, in_channels),
              nn.ReLU(),
              nn.Linear(in_channels, 1)
            )

    def forward(self, x):
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x
    
    


In [11]:
WrongResNetFinalLayersBlock(in_channels=512, seqsize=151)

TypeError: Can't instantiate abstract class WrongResNetFinalLayersBlock with abstract method train_step

FinalLayersBlock must implement loss calculation logic, so we have to define train_step (for other blocks the default implementation was used)


In [12]:
import torch
from typing import Any 

from prixfixe.prixfixe import FinalLayersBlock

class ResNetFinalLayersBlock(FinalLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         seqsize=seqsize)
        
        self.avgpool = nn.AvgPool1d(1) # global average pooling
        self.linear = nn.Sequential(
              nn.Linear(in_channels * self.seqsize, in_channels),
              nn.ReLU(),
              nn.Linear(in_channels, 1)
            )
        self.criterion = torch.nn.MSELoss()

    def forward(self, x):
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x
    
    def train_step(self, batch: dict[str, Any]) -> tuple[torch.Tensor, torch.Tensor]:
        x = batch["x"].to(self.device)
        y_pred = self.forward(x)
        y = batch["y"].to(self.device)
        loss = self.criterion(y, y_pred)
        
        return y, loss
    
    


In [13]:
ResNetFinalLayersBlock(in_channels=512, seqsize=151)

ResNetFinalLayersBlock(
  (avgpool): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (linear): Sequential(
    (0): Linear(in_features=77312, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
  (criterion): MSELoss()
)

# Joining blocks in PrixFixe

In [14]:
from prixfixe.prixfixe import PrixFixeNet

Pay attention to the fact that `FirstLayersBlock` and `CoreBlock` have the ".infer_outseqsize()” method to allow passing this information to the next blocks in the pipeline. This method is already implemented in abstract class so there is no need for teams to rewrite it 

In [15]:
first = ResnetFirstLayersBlock(in_channels=4,
                               out_channels=64,
                               seqsize=150)
core = ResNetCoreBlock(in_channels=first.out_channels,
                       out_channels=512,
                       seqsize=first.infer_outseqsize())
final = ResNetFinalLayersBlock(in_channels=core.out_channels,
                               seqsize=core.infer_outseqsize())
resnet = PrixFixeNet(first=first,
            core=core,
            final=final,
            generator=torch.Generator())

We can check the correctness of the final model

In [16]:
resnet.check()

Checking forward pass
Forward is OK
Checking training step
Training step is OK


And check if it is compatible with blocks from other teams.

In [17]:
from prixfixe.autosome import AutosomeCoreBlock

first = ResnetFirstLayersBlock(in_channels=4,
                               out_channels=64,
                               seqsize=150)
core =  AutosomeCoreBlock(in_channels=first.out_channels,
                       out_channels=512,
                       seqsize=first.infer_outseqsize())
final = ResNetFinalLayersBlock(in_channels=core.out_channels,
                               seqsize=core.infer_outseqsize())
resnet_with_autosome_core_block = PrixFixeNet(first=first,
            core=core,
            final=final,
            generator=torch.Generator())

In [18]:
resnet_with_autosome_core_block .check()

Checking forward pass
Forward is OK
Checking training step
Training step is OK
